---
format:
  html:
    embed-resources: true
    fig-width: 9
    fig-height: 6
jupyter: python3
code-fold: true
code-overflow: wrap
---

In [83]:
import numpy as np
from scipy.linalg import solve, lu, qr, cholesky, eig, svd
from scipy.sparse import csr_matrix
import scipy.sparse.linalg as spla
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import cg  # Conjugate Gradient

# Computational approaches

## NumPy and SciPy linear algebra solvers

### NumPy

NumPy provides basic linear algebra routines through $\texttt{numpy.linalg}$. These functions internally use **LAPACK** and **BLAS** for efficiency.


#### 1. **Solving a system of linear equations**  
   Given the system:

   $$ Ax = b $$

In [84]:
A = np.array([[3, 2], [1, 4]])
B = np.array([5, 6])
x = np.linalg.solve(A, B)  # Solves Ax = B
print(x)

[0.8 1.3]


This function uses an efficient algorithm (Gaussian elimination with partial pivoting) to find x. Refer to [this link](https://numpy.org/doc/2.2/reference/generated/numpy.linalg.solve.html) for more information.

#### 2. **Computing the inverse of a matrix**
  $$ A^{-1} $$

In [85]:
A_inv = np.linalg.inv(A) # Inverse of A
x = np.dot(A_inv, B) # Solves Ax = B
print(x)

[0.8 1.3]


Note that the matrix must non-singular $(\det(A) \neq 0)$  for this to work. Click the [link](https://numpy.org/doc/2.2/reference/generated/numpy.linalg.inv.html) for more information.

#### 3. **Computing the determinant**<br>
The determinant of a square matrix A is a scalar value that provides important information about the matrix. It is denoted as:
   $$ \det(A) $$

In [86]:
det_A = np.linalg.det(A) # Determinant of A
# Check if the determinant is zero
if det_A == 0:
    print("The system has no unique solution (determinant is zero).")
else:
    x = np.linalg.solve(A, B) # Solves Ax = B
    print(x)

[0.8 1.3]


A nonzero determinant means A is invertible, while det(A) = 0 means A is singular (not invertible).

#### 4. **Eigenvalues and Eigenvectors**

For a square matrix \( $A$ \), the eigenvalues (\( $\lambda$ \)) and corresponding eigenvectors (\( $v$ \)) satisfy the equation:

$$
A \cdot v = \lambda \cdot v
$$

where:

- \( $\lambda$ \) (lambda) is a **scalar** (eigenvalue),
- \( $v$ \) is a **nonzero vector** (eigenvector).

In [87]:
# Eigenvalues and eigenvectors
eigenvalues, eigenvectors  = np.linalg.eig(A) # compute eigenvalues and eigenvectors

# Diagonal matrix of eigenvalues
D = np.diag(eigenvalues)

# Compute P inverse
P_inv = np.linalg.inv(eigenvectors)

# Transform B into the eigenbasis
B_prime = P_inv @ B

# Solve for Y in DY = B'
Y = np.linalg.solve(D, B_prime)

# Compute the final solution X = P Y
X = eigenvectors @ Y

# Print the result
print("Solution X:", X)

Solution X: [0.8 1.3]


ELI5: Eigenvectors "point" in the same direction as most of your data, and eigenvalues tell you how strongly your data points that direction. 

#### 5. **Singular Value Decomposition (SVD)**

Singular Value Decomposition (SVD) is a method to break down a complex matrix into simpler components. It is similar to taking a blurry image and separating it into clear building blocks.

For any matrix \( $A$ \), SVD expresses it as:

$$ A = U \Sigma V^T $$

Where:
- **\( $U$ \)** (Left singular vectors) → Contains important patterns from the original data.
- **\( $\Sigma$ \)** (Singular values) → A diagonal matrix with values indicating the importance of each pattern.
- **\( $V^{T}$ \)** (Right singular vectors) → Contains another set of patterns that, when combined with \( $\Sigma$ \), recreate the original data.

Analogy:<br>
Imagine you have a large playlist of songs. SVD breaks it down into:
- **\( $U$\)** → The general themes of music (e.g., rock, jazz, pop).
- **\( $\Sigma$ \)** → How strongly each theme appears in the playlist.
- **\( $V^{T}$ \)** → The details of each song, arranged by themes.

In [88]:
# Compute Singular Value Decomposition (SVD)
U, S, V = np.linalg.svd(A)

# Convert S into a diagonal matrix
S_diag = np.diag(S)

# Compute the pseudo-inverse of S
S_inv = np.linalg.inv(S_diag)

# Compute the pseudo-inverse of A using SVD
A_pinv = V.T @ S_inv @ U.T # @ means matrix multiplication and .T means transpose

# Solve for X using the pseudo-inverse
X = A_pinv @ B

# Print the result
print("Solution X:", X)

Solution X: [0.8 1.3]


### SciPy

SciPy extends NumPy's linear algebra capabilities with additional **matrix decompositions, advanced solvers, and performance optimizations**.
Key Advantages of SciPy over NumPy:<br>
- Uses **LAPACK** and **ATLAS**, but with extra options for optimization.
- Supports sparse matrices via \texttt{scipy.sparse.linalg}.
- Includes additional decomposition methods.

#### 1. **Solving a linear system ($\texttt{scipy.linalg.solve}$)**  
   Same as $\texttt{numpy.linalg.solve}$ but often faster and more stable.


In [89]:
x = solve(A, B)  # More efficient than np.linalg.solve for large matrices
print(x)

[0.8 1.3]


#### 2. **LU Decomposition**  

   $$ PA = LU $$

In [90]:
P, L, U = lu(A)  # P: Permutation, L: Lower triangular, U: Upper triangular

# Compute Pb
Pb = np.dot(P, B)

# Solve Ly = Pb using forward substitution
y = np.linalg.solve(L, Pb)

# Solve Ux = y using backward substitution
x = np.linalg.solve(U, y)
x

array([0.8, 1.3])

#### 3. **QR Decomposition**  

   $$ A = QR $$

In [91]:
Q, R = qr(A)
# Compute Q^T B
QTB = np.dot(Q.T, B)

# Solve Rx = Q^T B using backward substitution
x = np.linalg.solve(R, QTB)
x

array([0.8, 1.3])

#### 4. **Cholesky Decomposition (for positive definite matrices)**  

Cholesky Decomposition is a matrix factorization technique used for symmetric positive definite matrices. Given a symmetric positive definite matrix \( A \), it can be decomposed as:

$$
A = LL^T
$$

where:
- \( $L$ \) is a **lower triangular matrix** with positive diagonal entries.
- \( $L^{T}$ \) is the **transpose** of \( L \), making the decomposition efficient and numerically stable.<br>
\
**Why Use Cholesky Decomposition?**<br>
1. **Efficient Computation**: It is about **twice as fast** as LU decomposition for solving linear systems.
2. **Numerical Stability**: Since it is applied only to positive definite matrices, it avoids issues related to pivoting.
3. **Applications**:
   - Solving linear systems \( $Ax = b$ \) efficiently.
   - Optimization problems (e.g., least squares, Kalman filters).
   - Monte Carlo simulations (e.g., generating correlated random variables). Helpful in Quantum Mechanical simulations.

**Algorithm for Cholesky Decomposition**
Given a symmetric positive definite matrix \( $A$ \), the elements of \( $L$ \) are computed as:

1. **Diagonal elements**:

$$
L_{ii} = \sqrt{A_{ii} - \sum_{k=1}^{i-1} L_{ik}^2}
$$

2. **Off-diagonal elements**:

$$
L_{ij} = \frac{1}{L_{ii}} \left( A_{ij} - \sum_{k=1}^{i-1} L_{ik} L_{jk} \right), \quad j > i
$$

To solve for \( x \) in the equation:

$$ Ax = B $$

using Cholesky decomposition, follow these steps:

**Steps for Solving \( $Ax = B$ \) Using Cholesky Decomposition**

1. **Check if \( $A$ \) is symmetric positive definite**
   - \( $A$ \) must be symmetric (\( $A = A^{T}$ \)).
   - All its eigenvalues must be positive.

2. **Perform Cholesky Decomposition**
   - Decompose \( $A$ \) as \( $A = LL^{T}$ \), where \( $L$ \) is a lower triangular matrix.

3. **Solve for \( $y$ \) in \( $Ly = B$ \)** (using forward substitution).

4. **Solve for \( $x$ \) in \( $L^{T} x = y$ \)** (using backward substitution).


In [92]:
L = cholesky(A, lower=True)
# Solve Ly = B using forward substitution
y = np.linalg.solve(L, B)

# Solve L^T x = y using backward substitution
x = np.linalg.solve(L.T, y)
x

array([1.27272727, 1.18181818])

The answer is different, issue coming from the fact that Cholesky decomposition only works for symmetric positive definite (SPD) matrices and the example above is **NOT** symmetric.

### **Example**
Consider the matrix:

$$
A =
\begin{bmatrix}
4 & 12 & -16 \\
12 & 37 & -43 \\
-16 & -43 & 98
\end{bmatrix}
$$

Applying Cholesky decomposition, we get:

$$
L =
\begin{bmatrix}
2 & 0 & 0 \\
6 & 1 & 0 \\
-8 & 5 & 3
\end{bmatrix}
$$

where \( $L^{T}$ \) is its transpose.

In [93]:
C = np.array([
    [4, 12, -16],
    [12, 37, -43],
    [-16, -43, 98]
])
L = cholesky(C, lower=True)
L

array([[ 2.,  0.,  0.],
       [ 6.,  1.,  0.],
       [-8.,  5.,  3.]])

and if you want to solve for x, assuming that B
$$
B =
\begin{bmatrix}
2  \\
6 \\
-8
\end{bmatrix}
$$


In [94]:
# Define vector B
D = np.array([2, 6, -8]) # B is being already above

# Perform Cholesky decomposition: A = L * L^T
L = cholesky(C, lower=True)

# Solve for y in L * y = B using forward substitution
y = np.linalg.solve(L, D)

# Solve for x in L^T * x = y using backward substitution
x = np.linalg.solve(L.T, y)

print("Solution x:", x)

Solution x: [0.5 0.  0. ]


#### 5. **Eigenvalue Problems ($\texttt{scipy.linalg.eig}$)**

ELI5, think of a spring:
- if you pull it straight, it stretches along its length—that’s like an eigenvector (a special direction).
- How much it stretches (double, triple, or shrinks) is the eigenvalue (a number describing the effect).

In [95]:
eigvals, eigvecs = eig(A)

# Compute V^(-1) * B
V_inv_B = np.linalg.solve(eigvecs, B)

# Solve D * y = V_inv_B (since D is diagonal, just divide)
y = V_inv_B / eigvals

# Compute final solution x = V * y
x = np.dot(eigvecs, y).real
x

array([0.8, 1.3])

Another example, array C from the example above

In [96]:
eigvals_c, eigvecs_c = eig(C)

# Compute V^(-1) * B
V_inv_D = np.linalg.solve(eigvecs_c, D)

# Solve D * y = V_inv_B (since D is diagonal, just divide)
y = V_inv_D / eigvals_c

# Compute final solution x = V * y  
x = np.dot(eigvecs_c, y).real.round(2) # real to remove imaginary part and round to 2 decimal places
x

array([ 0.5, -0. ,  0. ])

#### 6. **Singular Value Decomposition (SVD)**  
Singular Value Decomposition (SVD) is a way to break down a big, complicated matrix into smaller, simpler parts. For any matrix \( $A$ \), SVD expresses it as:
$$
A = U \Sigma V^T
$$
Where:
- **\( $U$ \)** (Left singular vectors) → Contains important patterns from the original data.
- **\( $\Sigma$ \)** (Singular values) → A diagonal matrix with values indicating the importance of each pattern.
- **\( $V^{T}$ \)** (Right singular vectors) → Contains another set of patterns that, when combined with \( \Sigma \), recreate the original data.


Analogy:<br>
Imagine you have a large playlist of songs. SVD breaks it down into:
- **\( $U$ \)** → The general themes of music (e.g., rock, jazz, pop).
- **\( $\Sigma$ \)** → How strongly each theme appears in the playlist.
- **\( $V^{T}$ \)** → The details of each song, arranged by themes.

In [97]:
U, S, V = svd(A)
# Compute the pseudo-inverse of Sigma
S_inv = np.diag(1 / S)

# Solve for x using the SVD components
x = V.T @ S_inv @ U.T @ B

print("Solution x:", x)

Solution x: [0.8 1.3]


Other example:

In [98]:
U1, S1, V1 = svd(C)
# Compute the pseudo-inverse of Sigma
S_inv1 = np.diag(1 / S1)

# Solve for x using the SVD components
x1 = V1.T @ S_inv1 @ U1.T @ D

print("Solution x:", x1.real.round(2))

Solution x: [ 0.5  0.  -0. ]


### Sparse Linear Algebra ($\texttt{scipy.sparse.linalg}$)
For large, sparse matrices, SciPy provides $\texttt{scipy.sparse.linalg}$, which is optimized for efficiency. if you don't know, A sparse matrix is a matrix in which most of the elements are zero. It is the opposite of a dense matrix, where most elements are nonzero.

#### 1. **Solving a sparse linear system**  

   $$ Ax = b $$
where $A$ is a sparse matrix.<br>
for example:

In [99]:
# sparse matrix A (5x5)
E = csr_matrix([
    [4, 1, 0, 0, 0],
    [1, 4, 1, 0, 0],
    [0, 1, 4, 1, 0],
    [0, 0, 1, 4, 1],
    [0, 0, 0, 1, 3]
])

# right-hand side vector b
F = np.array([1, 2, 3, 4, 5])

# Solve Ax = b
x = spla.spsolve(E, F)

# Print the solution
print("Solution x:", x)

Solution x: [0.16987741 0.32049037 0.54816112 0.48686515 1.50437828]


In [100]:
A_sparse = csc_matrix(A)  # Convert A to sparse format
x = spsolve(A_sparse, B)

#### 2. **Iterative solvers (Conjugate Gradient, GMRES)**  
Iterative solvers like **Conjugate Gradient (CG)** and **Generalized Minimal Residual (GMRES)** are widely used for solving large, sparse linear systems of the form:

$$
Ax = b
$$

where \( $A$ \) is a matrix, \( $x$ \) is the unknown vector, and \( $b$ \) is a given right-hand-side vector. These solvers are particularly useful when direct methods (like LU decomposition) are too expensive in terms of memory and computation.


**Comparison: CG vs. GMRES**


| Feature               | Conjugate Gradient (CG)            | GMRES                                      |
|-----------------------|-----------------------------------|--------------------------------------------|
| **Matrix Type**       | Symmetric Positive Definite      | General (Non-Symmetric, Non-SPD)          |
| **Storage**          | Low (only need a few vectors)    | High (grows with iterations)              |
| **Convergence**      | Faster for well-conditioned SPD matrices | Can stagnate without restarts    |
| **Computational Cost** | Cheaper per iteration           | More expensive due to orthogonalization   |
| **Restarting Needed?** | No                              | Yes (for large problems, GMRES(m))        |


In [102]:
# sparse matrix A (5x5)
E = csr_matrix([
    [4, 1, 0, 0, 0],
    [1, 4, 1, 0, 0],
    [0, 1, 4, 1, 0],
    [0, 0, 1, 4, 1],
    [0, 0, 0, 1, 3]
])

# right-hand side vector b
F = np.array([1, 2, 3, 4, 5])

# Solve

x, info = cg(E, F)
# Print the solution
print("Solution x:", x)
print("CG Info:", info) 

Solution x: [0.16987741 0.32049037 0.54816112 0.48686515 1.50437828]
CG Info: 0


info=0 means that the solution converged successfully.